In [1]:
from osgeo import ogr
from osgeo import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
import critical_loads as cl
import nivapy

# Connect to db
engine = nivapy.da.connect()

Username: ········
Password: ········
Connection successful.


# Critical Loads Workflow

This notebook implements a new workflow for the Critical Loads processing. There are three main components: vegetation, water and soils.

**Note:** In 2017, NILU switched to estimating deposition using a new method. For the period from 2012-2016, values will be calculated using both methods, and in future only the new method will be used. The data provided by NILU for the new method is in a slightly different format to what has been supplied previously. The code below is correct for the new method, but some minor modifications in section 1.1 would be required to go back to the old method.

## 1. Vegetation

A new raster-based workflow was developed in [this notebook](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_vegetation.ipynb). The code below loops over all the datasets to create vegetation exceedance grids (in mg-N/l) for the time periods of interest.

### 1.1. Upload new data to database

In [2]:
# ID for new series
ser_id = 27

# Create new series for this data period
df = pd.DataFrame({'dep_series_id':ser_id,
                   'name':'Middel 2012-2016 (new)',
                   'description':'Fordelt til BLR av NILU 2017 (Wenche Aas; new method)'},
                  index=[0,])

## Add to db
#df.to_sql('dep_series_definitions', con=engine, schema='resa2', 
#          if_exists='append', index=False)

In [3]:
# Read data
in_xlsx = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\raw_data\dep1216_til_NIVA_new_method.xlsx')
df = pd.read_excel(in_xlsx, sheetname='niva16x16', index_col=0)
df.index.name = 'blr'

# Map headings to db par IDs
par_dict = {'totSox':4,  # Non-marine S (I think - values are much closer to NM-S from old method than total-S)
            'totNox':1,  # N (oks)
            'totNhx':2}  # N (red)

# Get cols and rename
df = df[par_dict.keys()]
df.columns = [par_dict[i] for i in df.columns]
df.reset_index(inplace=True)

# Melt 
df = pd.melt(df, var_name='parameter_id', id_vars='blr')

# Add series ID
df['dep_series_id'] = ser_id

## Add to db
#df.to_sql('dep_blr_values', con=engine, schema='resa2', 
#          if_exists='append', index=False)

### 1.2. Read lookup table link vegetation classes to critical loads

In [4]:
# Read lookup table
in_xlsx = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\vegetation'
           r'\sat_veg_land_use_classes.xlsx')
df = pd.read_excel(in_xlsx, sheetname='EUNIS_tilGIS', index_col=0)

df = df[['CL_100smgN/m2/yr']].round(0).astype(int)

df.head()

,CL_100smgN/m2/yr
NORUTcode,
1,5
2,5
3,5
4,10
5,10


### 1.3. Reclassify

In [5]:
## Input national veg map
#in_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
#          r'\GIS\Raster\sat_veg_30m_all.tif')
#
## Output geotiff for critical loads values
#out_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
#           r'\GIS\Raster\sat_veg_30m_cr_lds_div100.tif')
#
## Mask raster for land defined by BLR
#mask_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
#            r'\GIS\Raster\blr_land_mask.tif')
#
## Reclassify
#cl.reclassify_raster(in_tif, mask_tif, out_tif, df, 'CL_100smgN/m2/yr', 255)

### 1.4. Get deposition data from database

**Note:** Remember to add the new `dep_series_id` to the query below, as well as a new name in `df.columns`.

In [2]:
# Get all dep values
sql = ("SELECT blr, dep_series_id as series, value as dep "
       "FROM resa2.dep_blr_values "
       "WHERE parameter_id IN (1, 2) "
       "AND dep_series_id = 27")

df = pd.read_sql(sql, engine)

# Sum N_oks and N_red
df = df.groupby(['blr', 'series']).sum()

# Reshape and tidy
df = df.unstack()
df.columns = df.columns.get_level_values(1)
df.columns.name = ''
df.columns = ['Ndep1216_2'] # '2' means 'new method'. Must be <=10 chars for shp

print np.nanmin(df.values), np.nanmax(df.values)

df.head()

62.07 1711.05


,Ndep1216_2
blr,
58006001,1057.03
58006002,1172.59
58006003,1229.18
58006004,1164.11
58006005,1050.53


### 1.5. Loop over datasets

In [3]:
%%time

# Path to raw BLR shapefile
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\blrgrid_uten_grums_utm_z33n.shp')

# Snap raster
snap_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
            r'\GIS\Raster\sat_veg_30m_all.tif')

# Critical loads raster
cl_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_30m_cr_lds_div100.tif')

# Container for output
data_dict = {'series':[],
             'nor_area_km2':[],
             'ex_area_km2':[]}

# Loop over series
for ser in df.columns:    
    
    print 'Processing: %s' % ser
    print '    Building deposition shapefile...'
    
    # Get deposition
    dep_df = df[[ser]].dropna(how='any').round(0).astype(int).reset_index()

    # Rename cols to match shapefile
    dep_df.columns = ['BLR', ser]

    # Read shapefile
    blr_df = gpd.read_file(in_shp)

    # Join and tidy
    dep_df = blr_df.merge(dep_df, on='BLR')
    del dep_df['area_m2']
    
    # Write output shapefile
    dep_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
               r'\Shapefiles\dep_%s.shp' % ser)
    dep_df.to_file(dep_shp)
    
    # Convert shp to ras
    print '    Rasterising shapefile...'
    
    # Output BLR raster
    dep_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
               r'\GIS\Raster\dep_%s_30m.tif' % ser)

    cl.vec_to_ras(dep_shp, dep_tif, snap_tif, ser, -1, gdal.GDT_Int16)
    
    # Exceedance
    print '    Calculating exceedance...'
    
    # Read grids
    cl_grid, cl_ndv = cl.read_geotiff(cl_tif)
    dep_grid, dep_ndv = cl.read_geotiff(dep_tif)

    # Upcast to float32 for safe handling of negative values
    cl_grid = cl_grid.astype(np.float32)
    dep_grid = dep_grid.astype(np.float32)
   
    # Set ndv
    cl_grid[cl_grid==cl_ndv] = np.nan
    dep_grid[dep_grid==dep_ndv] = np.nan

    # Get total area of non-NaN from dep grid
    nor_area = np.count_nonzero(~np.isnan(dep_grid))*30.*30./1.E6

    # Apply scaling factor to CLs
    cl_grid = cl_grid*100.

    # Exceedance
    ex_grid = dep_grid - cl_grid
    del dep_grid, cl_grid  
    
    # Get total area exceeded
    ex_area = np.count_nonzero(ex_grid > 0)*30.*30./1.E6

    # Set <0 to 0
    ex_grid[ex_grid<0] = 0
    
    # Reset ndv
    ex_grid[np.isnan(ex_grid)] = -1

    # Downcast to int16 to save space
    ex_grid = ex_grid.round(0).astype(np.int16)
    
    # Append results
    data_dict['series'].append(ser)
    data_dict['nor_area_km2'].append(nor_area)
    data_dict['ex_area_km2'].append(ex_area)
    
    # Write output
    print '    Saving exceedance grid...'
    ex_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
              r'\GIS\Raster\exceed_%s_30m.tif' % ser)
    
    cl.write_geotiff(ex_grid, ex_tif, snap_tif, -1, gdal.GDT_Int16)
    
    print '    Done.'

# Build output df
ex_df = pd.DataFrame(data_dict)
ex_df['ex_pct'] = 100 * ex_df['ex_area_km2'] / ex_df['nor_area_km2']
ex_df.index = ex_df['series']
del ex_df['series']
ex_df = ex_df.round(0).astype(int)

# Save
out_csv = r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\vegetation\nor_prop_exceed.csv'
ex_df.to_csv(out_csv)

print 'Finished.'

Processing: Ndep1216_2
    Building deposition shapefile...
    Rasterising shapefile...


C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:78: RuntimeWarning: invalid value encountered in greater
C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:81: RuntimeWarning: invalid value encountered in less


    Calculating exceedance...
    Saving exceedance grid...
    Done.
Finished.
Wall time: 2min 4s


### 1.6. Print national exceedance summary for vegetation

In [4]:
# Exceedance summary
ex_df

,ex_area_km2,nor_area_km2,ex_pct
series,,,
Ndep1216_2,64001,320584,20


## 2. Water

The new methodology for water exceedances is developed [here](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_water.ipynb). The code below is copied from this notebook - see the original for further explanation.

### 2.1. Extract deposition data

**Note:** Remember to add the new `dep_series_id` to the query below.

In [11]:
# Get dep data
sql = ("SELECT a.blr, b.name AS series, c.name AS par, a.value AS dep "
       "FROM resa2.dep_blr_values a, "
       "resa2.dep_series_definitions b, "
       "resa2.air_parameter_definitions c "
       "WHERE a.parameter_id IN (1, 2, 4) "
       "AND a.dep_series_id = 27 "
       "AND a.parameter_id = c.parameter_id "
       "AND a.dep_series_id = b.dep_series_id")

dep_df = pd.read_sql(sql, engine)

# Tidy
dep_df['series'] = dep_df['series'].str[7:]
dep_df['par'] = dep_df['par'].str[:1]

# Group N(oks) and N(red)
dep_df = dep_df.groupby(['blr', 'series', 'par']).sum()

# Unstack
dep_df = dep_df.unstack('par')
dep_df.columns = dep_df.columns.get_level_values('par')

# Convert to meq/l
dep_df['N'] = dep_df['N'] / 14.
dep_df['S'] = dep_df['S']*2. / 32.06

# Unstack
dep_df = dep_df.unstack('series')

# Flatten col index
dep_df.columns = ['%s_%s' % (p, d) for p, d in zip(dep_df.columns.get_level_values('par'), 
                                                   dep_df.columns.get_level_values('series'))]

dep_df.head()

,N_2012-2016 (new),S_2012-2016 (new)
blr,,
58006001,75.502143,19.305677
58006002,83.756429,21.245789
58006003,87.798571,23.014972
58006004,83.150714,22.127885
58006005,75.037857,20.973799


### 2.2. Get critical loads

In [12]:
# Get dep data
sql = ("SELECT blr, claoaavaroaa, eno3fl, clminn, clmaxnoaa, clmaxsoaa "
       "FROM resa2.cla")

cl_df = pd.read_sql(sql, engine)

# Set index
cl_df.index = cl_df['blr']
del cl_df['blr']

# Add CLSmin as 0
cl_df['clmins'] = 0

cl_df.head()

,claoaavaroaa,eno3fl,clminn,clmaxnoaa,clmaxsoaa,clmins
blr,,,,,,
60506013,46.689055,20.499,3.204124,59.153507,47.083568,0
60508003,47.240467,8.988,3.204124,70.544508,48.610725,0
60509002,48.396218,0.029,14.635616,100.955383,51.368535,0
60511002,11.156834,0.080,7.077605,27.656760,11.903564,0
60511006,34.764804,1.081,42.316831,121.992502,38.498971,0


### 2.3. Calculate exceedances

#### 2.3.1. Check data and join

In [13]:
# Check dfs are compatible
assert len(dep_df) == len(cl_df), 'Lengths of dataframes do not match.'
assert dep_df.index.is_unique, 'dep_df has duplicated BLR IDs.'
assert cl_df.index.is_unique, 'cl_df has duplicated BLR IDs.'

# Join
df = dep_df.join(cl_df, how='left')

# Fill NaN with 0
df.fillna(0, inplace=True)

df.head()

,N_2012-2016 (new),S_2012-2016 (new),claoaavaroaa,eno3fl,clminn,clmaxnoaa,clmaxsoaa,clmins
blr,,,,,,,,
58006001,75.502143,19.305677,28.533538,35.544,3.204124,40.514435,29.056767,0
58006002,83.756429,21.245789,32.029772,27.861,3.204124,43.473268,32.471152,0
58006003,87.798571,23.014972,20.888186,25.137,3.204124,29.564269,21.184960,0
58006004,83.150714,22.127885,18.914393,36.846,25.972633,49.911208,19.189395,0
58006005,75.037857,20.973799,20.463044,47.587,3.204124,30.057701,20.846989,0


#### 2.3.2. SSWC model

**Note:** Remember to update the `periods` list below.

In [14]:
# Define time periods
periods = ['2012-2016 (new)',]

# Copy df
sswc_df = df.copy()

# Loop over periods
for per in periods:
    sswc_df['sswc_ex_%s' % per] = (sswc_df['S_%s' % per] + 
                                   sswc_df['eno3fl'] - 
                                   sswc_df['claoaavaroaa'])

# Get cols of interest
cols = ['sswc_ex_%s' % per for per in periods]
sswc_df = sswc_df[cols]

# Set values below 0 to 0
sswc_df[sswc_df<0] = 0

# Write to output
out_path = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water\sswc_exceedance_12-16_new.csv')
sswc_df.to_csv(out_path)

sswc_df.head()

,sswc_ex_2012-2016 (new)
blr,
58006001,26.316139
58006002,17.077017
58006003,27.263786
58006004,40.059492
58006005,48.097755


#### 2.3.3. FAB model

**Note:** Remember to update the `periods` list below.

In [15]:
# Define time periods
periods = ['2012-2016 (new)',]

# Plot folder
png_fold = r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water\CLF_PNGs'

# Copy df
fab_df = df.copy()

# Loop over periods
for per in periods:
    print 'Processing %s...' % per
    
    # Containers for data
    fab_ex_n = []
    fab_ex_s = []
    fab_ex_reg = []
    
    # Loop over rows
    for idx, row in df.iterrows():
        # Calc exceedance
        ex_n, ex_s, reg_id = cl.exceed_ns_icpm(row['clminn'], row['clmaxnoaa'],
                                               row['clmins'], row['clmaxsoaa'],
                                               row['N_%s' % per], row['S_%s' % per])
        
        # Add to result
        fab_ex_n.append(ex_n)
        fab_ex_s.append(ex_s)
        fab_ex_reg.append(reg_id)
        
        # Plot
#        out_png = os.path.join(png_fold, 'blr_%s_%s.png' % (idx, per))
#        cl.plot_critical_loads_func(row['clminn'], row['clmaxnoaa'],
#                                    row['clmins'], row['clmaxsoaa'],
#                                    [row['N_%s' % per],], [row['S_%s' % per]],
#                                    title='%s (%s)' % (idx, per), save_png=True, 
#                                    png_path=out_png)
#        plt.close()
        
    # Add to df
    fab_df['fab_ex_n_%s' % per] = fab_ex_n
    fab_df['fab_ex_s_%s' % per] = fab_ex_s
    fab_df['fab_ex_reg_%s' % per] = fab_ex_reg
    
    # Calc total ex = ex_n + ex_s
    fab_df['fab_ex_%s' % per] = fab_df['fab_ex_n_%s' % per] + fab_df['fab_ex_s_%s' % per]

# Get cols of interest
cols = [i for i in fab_df.columns if i[:3]=='fab']
fab_df = fab_df[cols]

# Set values below 0 to 0
fab_df[fab_df<0] = 0

# Write to output
out_path = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water\fab_exceedance_12-16_new.csv')
fab_df.to_csv(out_path)

print 'Finished.'

fab_df.head()

Processing 2012-2016 (new)...
Finished.


,fab_ex_n_2012-2016 (new),fab_ex_s_2012-2016 (new),fab_ex_reg_2012-2016 (new),fab_ex_2012-2016 (new)
blr,,,,
58006001,34.987708,19.305677,2,54.293384
58006002,40.283160,21.245789,2,61.528949
58006003,58.234303,23.014972,2,81.249275
58006004,33.239506,22.127885,2,55.367391
58006005,44.980157,20.973799,2,65.953956


#### 2.3.4. Area exceeded

In [16]:
# Read BLR grid
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\blrgrid_uten_grums.shp')
blr_df = gpd.read_file(in_shp)

# Convert to 'normal' df
del blr_df['geometry']
blr_df = pd.DataFrame(blr_df)

# Get total area of Norway by summing cells
nor_area = blr_df['area_m2'].sum()/1E6

blr_df.head()

,BLR,area_m2
0,58005004,5.410322e+07
1,58006001,1.939285e+08
2,58006002,2.044114e+08
3,58006003,2.043394e+08
4,58006004,2.042695e+08


##### 2.3.4.1. SSWC

In [17]:
# Container for output
data_dict = {'period':[],
             'area_exceeded_km2':[],
             'area_total_km2':[]}

# Caclulate area exceeded
for col in sswc_df.columns:
    per = col[-9:]
    
    # Get BLRs exceeded
    ex_blr = sswc_df[sswc_df[col]>0].index
    
    # Get area exceeded
    area_ex = blr_df['area_m2'][blr_df['BLR'].isin(ex_blr)].sum()/1E6
    
    # Add to output
    data_dict['period'].append(per)
    data_dict['area_exceeded_km2'].append(area_ex)
    data_dict['area_total_km2'].append(nor_area)
    
# Build df
sswc_areas = pd.DataFrame(data_dict)
sswc_areas.index = sswc_areas['period']
del sswc_areas['period']

# Pct exceed
sswc_areas['pct_exceed'] = 100*sswc_areas['area_exceeded_km2']/nor_area

# Tidy
sswc_areas = sswc_areas.round(0).astype(int)

# Write output
out_csv = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water'
           r'\sswc_area_exceeded.csv')
sswc_areas.to_csv(out_csv)

sswc_areas

,area_exceeded_km2,area_total_km2,pct_exceed
period,,,
016 (new),23466,322183,7


##### 2.3.4.2. FAB

**Note:** Remember to update the `periods` list below.

In [19]:
# Get cols for overall exceedance
periods = ['2012-2016 (new)',]
cols = ['fab_ex_%s' % i for i in periods]
fab = fab_df[cols]

# Container for output
data_dict = {'period':[],
             'area_exceeded_km2':[],
             'area_total_km2':[]}

# Caclulate area exceeded
for col in fab.columns:
    per = col[-9:]
    
    # Get BLRs exceeded
    ex_blr = fab[fab[col]>0].index
    
    # Get area exceeded
    area_ex = blr_df['area_m2'][blr_df['BLR'].isin(ex_blr)].sum()/1E6
    
    # Add to output
    data_dict['period'].append(per)
    data_dict['area_exceeded_km2'].append(area_ex)
    data_dict['area_total_km2'].append(nor_area)
    
# Build df
fab_areas = pd.DataFrame(data_dict)
fab_areas.index = fab_areas['period']
del fab_areas['period']

# Pct exceed
fab_areas['pct_exceed'] = 100*fab_areas['area_exceeded_km2']/nor_area

# Tidy
fab_areas = fab_areas.round(0).astype(int)

# Write output
out_csv = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\water'
           r'\fab_area_exceeded.csv')
fab_areas.to_csv(out_csv)

fab_areas

,area_exceeded_km2,area_total_km2,pct_exceed
period,,,
016 (new),60897,322183,19


## 3. Soil

The soil calculations use the "old" method - see e-mail from Kari received 01/11/2017 at 13.47. The equation is simply

$$E_{soil} = S_{dep} - CL_{soil}$$

### 3.1. Get deposition data from database

**Note:** Remember to add the new `dep_series_id` to the query below, as well as a new name in `df.columns`.

In [21]:
# Get all dep values for non-marine S
# in mg-S/m2/year
sql = ("SELECT blr, dep_series_id as series, value as dep "
       "FROM resa2.dep_blr_values "
       "WHERE parameter_id = 4 "
       "AND dep_series_id = 27")

dep_df = pd.read_sql(sql, engine)

# Reshape and tidy
dep_df = dep_df.pivot(index='blr', columns='series', values='dep')
dep_df.columns.name = ''
dep_df.columns = ['Sdep1216_2']

dep_df.head()

,Sdep1216_2
blr,
58006001,309.47
58006002,340.57
58006003,368.93
58006004,354.71
58006005,336.21


### 3.2. Get critical loads

**Note:** Parameter 86 (`CL-Soil-N`) is strangely named, but the description implies it is the critical load for S.

**Note 2:** There is one cell where the critical loads is negative. This is filtered out from the calculations - see e-mail from Espen received 16/11/2017 at 13.04.

In [22]:
# Get all CLs for S
# in meq/m2/year
sql = ("SELECT blr, xvalue as crit_ld "
       "FROM resa2.talegren_values "
       "WHERE talegren_paramid = 86")

cl_df = pd.read_sql(sql, engine)
cl_df.index = cl_df['blr']
del cl_df['blr']

# Convert to mg-S/m2/yr
cl_df['crit_ld'] = cl_df['crit_ld']*32.06 / 2.

# Remove negative CL
cl_df = cl_df.query('crit_ld >= 0')

cl_df.head()

,crit_ld
blr,
58006004,1818.923277
58006007,3237.448624
58006008,3095.109867
58006012,3443.772948
58006015,3062.583133


### 3.3. Calculate exceedances

In [23]:
# Join
df = dep_df.join(cl_df)

# Exceedance
for col in dep_df.columns:
    per = col[-5:]
    df['Ex%s' % per] = df[col] - df['crit_ld']
    
# Get cols of interest
ex_df = df[[i for i in df.columns if i[:2]=='Ex']].copy()
ex_df.dropna(how='any', inplace=True)

ex_df.head()

,Ex216_2
blr,
58006004,-1464.213277
58006007,-2838.098624
58006008,-2670.339867
58006012,-3005.022948
58006015,-2751.633133


### 3.4. Area exceeded

In [24]:
# Read BLR grid
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\blrgrid_uten_grums.shp')
blr_df = gpd.read_file(in_shp)

# Convert to 'normal' df
del blr_df['geometry']
blr_df = pd.DataFrame(blr_df)

# Get total area of Norway by summing cells
nor_area = blr_df['area_m2'].sum()/1E6

blr_df.head()

,BLR,area_m2
0,58005004,5.410322e+07
1,58006001,1.939285e+08
2,58006002,2.044114e+08
3,58006003,2.043394e+08
4,58006004,2.042695e+08


In [25]:
# Container for output
data_dict = {'period':[],
             'area_exceeded_km2':[],
             'area_total_km2':[]}

# Caclulate area exceeded
for col in ex_df.columns:
    per = col[-5:]
    
    # Get BLRs exceeded
    ex_blr = ex_df[ex_df[col]>0].index
    
    # Get area exceeded
    area_ex = blr_df['area_m2'][blr_df['BLR'].isin(ex_blr)].sum()/1E6
    
    # Add to output
    data_dict['period'].append(per)
    data_dict['area_exceeded_km2'].append(area_ex)
    data_dict['area_total_km2'].append(nor_area)
    
# Build df
areas = pd.DataFrame(data_dict)
areas.index = areas['period']
del areas['period']

# Pct exceed
areas['pct_exceed'] = 100*areas['area_exceeded_km2']/nor_area

# Tidy
areas = areas.round(2)

# Write output
out_csv = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\soils'
           r'\soils_area_exceeded.csv')
areas.to_csv(out_csv)

areas

,area_exceeded_km2,area_total_km2,pct_exceed
period,,,
216_2,0.0,322183.39,0.0
